In [1]:
import numpy as np
from os import path, listdir, makedirs
from itertools import product
from brainpipe.system import study
from brainpipe.feat.utils._feat import _manageWindow
from brainpipe.feature import phase
from mne.filter import resample

#### Compute all phase files for all configurations
theta freqs, smoothing and downsampling parameters

In [3]:
st = study('Olfacto')
reps = ['Encoding_By_Odor']

for rep in reps:
    path2save = path.join(st.path, 'feature/TPSim_phase_data/Phase_by_odor_all_elecsFT/')
    code = 'E' if rep.startswith('Encoding') else 'R'
    ###############################################################################
    if not path.exists(path2save):
        makedirs(path2save)
    ###############################################################################  
    files = [k for k in st.search('bipo_all_noWM_physFT.npz', folder=('database/'+rep+'/'))]
    for fi in files:
        # Dict with all power features
        kwargs = {} # Define an empty dictionnary to save all power parameters
        kwargs['f'] = [[3,7],[4,7],[3,8],[4,8]] # Frequency vector

        # Load file :
        loadname = path.join(st.path, 'database/'+rep+'/', fi)
        mat = np.load(loadname,allow_pickle=True)
        print('all files in archive', mat.files)
        x, sf = mat['x'], 512
        n_elec, n_pts, n_trials = x.shape
        print ('--> compute power on : ', rep +'/'+ fi,'shape', x.shape)

        # Compute and save phase objects :
        powObj_x = phase(sf, n_pts,**kwargs, width=None, step=None, method='hilbert')
        powObj_x1 = phase(sf, n_pts,**kwargs, width=6, step=6, method='hilbert')
        win_all, time = _manageWindow(x.shape[1], width=None, step=None)
        win_all, time_ds = _manageWindow(x.shape[1], width=6, step=6)
        xpow = powObj_x.get(x,n_jobs=-1)[0]
        xpow_1 = powObj_x1.get(x,n_jobs=-1)[0]
        xpow_down = resample(xpow,down=6,axis=-2)
        print('phase None', xpow.shape, 'phase_ds', xpow_down.shape, 'phase_smooth',xpow_1.shape)
        print('time',np.array(time).shape,'time ds', np.array(time_ds).shape)
        
        kwargs['labels'], kwargs['channels']= mat['Mai_RL'], mat['channels']
        kwargs['xyz'] = mat['xyz']
        
        kwargs['time'], kwargs['time_ds'] = np.array(time) / sf, np.array(time_ds) / sf 
        kwargs['phase_None'], kwargs['phase_ds'] = xpow, xpow_down
        kwargs['phase_smooth'] = xpow_1
        save_x = path2save+ fi.replace('bipo_all_noWM_physFT.npz', 'bipo_all_'+code+'_4theta.npz')
        np.savez(save_x, **kwargs)
        del kwargs['phase_ds'],  kwargs['phase_None'], kwargs['phase_smooth']
        del kwargs, x, sf, n_elec, n_trials

-> Olfacto loaded
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_1_bipo_all_noWM_physFT.npz shape (61, 3584, 1)
phase None (61, 4, 3584, 1) phase_ds (61, 4, 597, 1) phase_smooth (61, 4, 597, 1)
time (3584,) time ds (597,)
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_2_bipo_all_noWM_physFT.npz shape (61, 3584, 3)
phase None (61, 4, 3584, 3) phase_ds (61, 4, 597, 3) phase_smooth (61, 4, 597, 3)
time (3584,) time ds (597,)
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/LEFC_odor_14_bipo_all_noWM_physFT.npz shape (27, 3584, 6)
phase None (27, 4, 3584, 6) phase_ds (27, 4, 597, 6) phase_smooth (27, 4, 597, 6)
time (3584,) time ds (597,)
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA',

phase None (61, 4, 3584, 3) phase_ds (61, 4, 597, 3) phase_smooth (61, 4, 597, 3)
time (3584,) time ds (597,)
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_9_bipo_all_noWM_physFT.npz shape (61, 3584, 5)
phase None (61, 4, 3584, 5) phase_ds (61, 4, 597, 5) phase_smooth (61, 4, 597, 5)
time (3584,) time ds (597,)
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/FERJ_odor_5_bipo_all_noWM_physFT.npz shape (32, 3584, 3)
phase None (32, 4, 3584, 3) phase_ds (32, 4, 597, 3) phase_smooth (32, 4, 597, 3)
time (3584,) time ds (597,)
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/FERJ_odor_7_bipo_all_noWM_physFT.npz shape (32, 3584, 3)
phase None (32, 4, 3584, 3) phase_ds (32, 4, 597, 3) phase_smooth (32, 4, 597, 3)
time (3584,) time 

#### Concatenate power by cond (ds, smooth, None)

In [5]:
import numpy as np
from brainpipe.system import study
from utils import odor_groups_3wgth as dict_
from os.path import join, exists
from os import makedirs
from itertools import product

st = study('Olfacto')
PATH = join(st.path, 'feature/TPSim_phase_data/')
PATH_OD = join(PATH, 'Phase_by_odor_all_elecsFT/')
od_name = join(PATH_OD, '{}_odor_{}_bipo_all_E_4theta.npz')
PATH_COND = join(PATH, 'Phase_by_cond_3groups_v=1_elecs=all/')
save_name = join(PATH_COND, '{}_odor_{}_E_4theta.npz')
data_concat = ['phase_ds','phase_None','phase_smooth']

if not exists(PATH_COND):
    makedirs(PATH_COND)
    
for su in dict_:
    for cond in dict_[su]:
        print('>>> processing:', su, cond)
        all_pow = []
        for data_c in data_concat:
            pow_data = np.array([])
            for od in dict_[su][cond]:
                mat = np.load(od_name.format(su,od),allow_pickle=True)
                pow_data = np.concatenate((pow_data, mat[data_c]),axis=-1) \
                                                if np.size(pow_data) else mat[data_c]
            all_pow.append(pow_data)
            print(data_c, su, od, cond, pow_data.shape)
        dict_pow = {}
        for file in mat.files:
            dict_pow[file] = mat[file]
        for i,data_c in enumerate(data_concat):
            dict_pow[data_c] = all_pow[i].swapaxes(0,1)
        np.savez(save_name.format(su,cond),**dict_pow)
        print(dict_pow['phase_None'].shape)

-> Olfacto loaded
>>> processing: CHAF low
phase_ds CHAF 4 low (61, 4, 597, 5)
phase_None CHAF 4 low (61, 4, 3584, 5)
phase_smooth CHAF 4 low (61, 4, 597, 5)
(4, 61, 3584, 5)
>>> processing: CHAF mid
phase_ds CHAF 8 mid (61, 4, 597, 4)
phase_None CHAF 8 mid (61, 4, 3584, 4)
phase_smooth CHAF 8 mid (61, 4, 597, 4)
(4, 61, 3584, 4)
>>> processing: CHAF high
phase_ds CHAF 9 high (61, 4, 597, 12)
phase_None CHAF 9 high (61, 4, 3584, 12)
phase_smooth CHAF 9 high (61, 4, 597, 12)
(4, 61, 3584, 12)
>>> processing: VACJ low
phase_ds VACJ 14 low (39, 4, 597, 9)
phase_None VACJ 14 low (39, 4, 3584, 9)
phase_smooth VACJ 14 low (39, 4, 597, 9)
(4, 39, 3584, 9)
>>> processing: VACJ mid
phase_ds VACJ 17 mid (39, 4, 597, 4)
phase_None VACJ 17 mid (39, 4, 3584, 4)
phase_smooth VACJ 17 mid (39, 4, 597, 4)
(4, 39, 3584, 4)
>>> processing: VACJ high
phase_ds VACJ 15 high (39, 4, 597, 9)
phase_None VACJ 15 high (39, 4, 3584, 9)
phase_smooth VACJ 15 high (39, 4, 597, 9)
(4, 39, 3584, 9)
>>> processing: SEM

#### Compute between-TPD by cond

In [6]:
from itertools import product,combinations
from scipy import stats
from pingouin import circ_corrcc
"""
Compute TPSim by combining all odors from each CONDITION
"""
###############################################################################
st = study('Olfacto')
path_data = join(st.path, 'feature/TPSim_phase_data/Phase_by_cond_3groups_v=1_elecs=all/')
pow_file = join(path_data, '{}_odor_{}_E_4theta.npz')
pathsave = join(st.path,'feature/TPSim_3groups_Enc_phase/TPS_btw_v=1_elecs=all_ds_sm/')#_'+RT_type+'/')
savename = join(pathsave,'TPS_pears_{}_{}_btw_theta{}_data={}_win={}.npz')
###############################################################################
subjects = ['PIRJ','CHAF','FERJ','VACJ','SEMC','LEFC']
conds = ['low','mid','high']
data_concat = ['phase_ds','phase_None','phase_smooth']
wins = {'3s': [-1,2], '2s':[0,2]}

if not exists(pathsave):
    makedirs(pathsave)
    
def tpsim_by_cond(su,cond,data_type,win_type):
    print(su,cond,data_type,win_type)
    mat = np.load(pow_file.format(su,cond),allow_pickle=True)
    time = mat['time_ds']-3 if data_type in ['phase_ds','phase_smooth'] else mat['time']-3
    lims = wins[win_type]
    nsel = [i for i,t in enumerate(time) if lims[0]<=t<=lims[1]]
    print(nsel[0],nsel[-1],len(nsel))
    pow_data = mat[data_type][:,:,nsel,:] #3584 points
    _,nelecs,npts,ntrials = pow_data.shape
    for f in range(len(mat['f'])):
        data_freq = pow_data[f]
        R_freq, p_freq = np.array([]), np.array([])
        for elec in range(nelecs):
            pow0 = data_freq[elec,...]
            R_trials, p_trials = np.array([]), np.array([])
            for t0, t1 in combinations(np.arange(ntrials), 2):
                R, p = circ_corrcc(pow0[:,t0],pow0[:,t1])
                D = 1 - R # <<<<<<< HERE TO CHANGE FOR DISTANCE COMPUTATIONS
                R_trials = np.vstack((R_trials,D)) if np.size(R_trials) else D
                p_trials = np.vstack((p_trials,p)) if np.size(p_trials) else p
            R_freq = np.vstack((R_freq,R_trials.T)) if np.size(R_freq) else R_trials.T
            p_freq = np.vstack((p_freq,p_trials.T)) if np.size(p_freq) else p_trials.T
        #R_freq = np.arctanh(R_freq)
        print(su,cond,'theta',f,'TPSim',R_freq.shape, p_freq.shape,'initial data',pow_data.shape)
        dict_ = {'tps':R_freq, 'pval':p_freq,'label':mat['labels'],
                           'channel':mat['channels'], 'xyz':mat['xyz']}
        np.savez(savename.format(su,cond,str(f),data_type,win_type),**dict_)

for su,cond,data_type,win in product(subjects,conds,data_concat,wins):
    tpsim_by_cond(su,cond,data_type,win)
# Parallel(n_jobs=-1)(delayed(
#     tpsim_by_cond)(su,cond) for su,cond in product(subjects,conds))

-> Olfacto loaded
PIRJ low phase_ds 3s
171 426 256
PIRJ low theta 0 TPSim (18, 91) (18, 91) initial data (4, 18, 256, 14)
PIRJ low theta 1 TPSim (18, 91) (18, 91) initial data (4, 18, 256, 14)
PIRJ low theta 2 TPSim (18, 91) (18, 91) initial data (4, 18, 256, 14)
PIRJ low theta 3 TPSim (18, 91) (18, 91) initial data (4, 18, 256, 14)
PIRJ low phase_ds 2s
256 426 171
PIRJ low theta 0 TPSim (18, 91) (18, 91) initial data (4, 18, 171, 14)
PIRJ low theta 1 TPSim (18, 91) (18, 91) initial data (4, 18, 171, 14)
PIRJ low theta 2 TPSim (18, 91) (18, 91) initial data (4, 18, 171, 14)
PIRJ low theta 3 TPSim (18, 91) (18, 91) initial data (4, 18, 171, 14)
PIRJ low phase_None 3s
1024 2560 1537
PIRJ low theta 0 TPSim (18, 91) (18, 91) initial data (4, 18, 1537, 14)
PIRJ low theta 1 TPSim (18, 91) (18, 91) initial data (4, 18, 1537, 14)
PIRJ low theta 2 TPSim (18, 91) (18, 91) initial data (4, 18, 1537, 14)
PIRJ low theta 3 TPSim (18, 91) (18, 91) initial data (4, 18, 1537, 14)
PIRJ low phase_None 2s

CHAF mid theta 3 TPSim (61, 6) (61, 6) initial data (4, 61, 171, 4)
CHAF mid phase_None 3s
1024 2560 1537
CHAF mid theta 0 TPSim (61, 6) (61, 6) initial data (4, 61, 1537, 4)
CHAF mid theta 1 TPSim (61, 6) (61, 6) initial data (4, 61, 1537, 4)
CHAF mid theta 2 TPSim (61, 6) (61, 6) initial data (4, 61, 1537, 4)
CHAF mid theta 3 TPSim (61, 6) (61, 6) initial data (4, 61, 1537, 4)
CHAF mid phase_None 2s
1536 2560 1025
CHAF mid theta 0 TPSim (61, 6) (61, 6) initial data (4, 61, 1025, 4)
CHAF mid theta 1 TPSim (61, 6) (61, 6) initial data (4, 61, 1025, 4)
CHAF mid theta 2 TPSim (61, 6) (61, 6) initial data (4, 61, 1025, 4)
CHAF mid theta 3 TPSim (61, 6) (61, 6) initial data (4, 61, 1025, 4)
CHAF mid phase_smooth 3s
171 426 256
CHAF mid theta 0 TPSim (61, 6) (61, 6) initial data (4, 61, 256, 4)
CHAF mid theta 1 TPSim (61, 6) (61, 6) initial data (4, 61, 256, 4)
CHAF mid theta 2 TPSim (61, 6) (61, 6) initial data (4, 61, 256, 4)
CHAF mid theta 3 TPSim (61, 6) (61, 6) initial data (4, 61, 256

FERJ high theta 1 TPSim (32, 36) (32, 36) initial data (4, 32, 1025, 9)
FERJ high theta 2 TPSim (32, 36) (32, 36) initial data (4, 32, 1025, 9)
FERJ high theta 3 TPSim (32, 36) (32, 36) initial data (4, 32, 1025, 9)
FERJ high phase_smooth 3s
171 426 256
FERJ high theta 0 TPSim (32, 36) (32, 36) initial data (4, 32, 256, 9)
FERJ high theta 1 TPSim (32, 36) (32, 36) initial data (4, 32, 256, 9)
FERJ high theta 2 TPSim (32, 36) (32, 36) initial data (4, 32, 256, 9)
FERJ high theta 3 TPSim (32, 36) (32, 36) initial data (4, 32, 256, 9)
FERJ high phase_smooth 2s
256 426 171
FERJ high theta 0 TPSim (32, 36) (32, 36) initial data (4, 32, 171, 9)
FERJ high theta 1 TPSim (32, 36) (32, 36) initial data (4, 32, 171, 9)
FERJ high theta 2 TPSim (32, 36) (32, 36) initial data (4, 32, 171, 9)
FERJ high theta 3 TPSim (32, 36) (32, 36) initial data (4, 32, 171, 9)
VACJ low phase_ds 3s
171 426 256
VACJ low theta 0 TPSim (39, 36) (39, 36) initial data (4, 39, 256, 9)
VACJ low theta 1 TPSim (39, 36) (39, 

SEMC low theta 0 TPSim (53, 210) (53, 210) initial data (4, 53, 171, 21)
SEMC low theta 1 TPSim (53, 210) (53, 210) initial data (4, 53, 171, 21)
SEMC low theta 2 TPSim (53, 210) (53, 210) initial data (4, 53, 171, 21)
SEMC low theta 3 TPSim (53, 210) (53, 210) initial data (4, 53, 171, 21)
SEMC mid phase_ds 3s
171 426 256
SEMC mid theta 0 TPSim (53, 45) (53, 45) initial data (4, 53, 256, 10)
SEMC mid theta 1 TPSim (53, 45) (53, 45) initial data (4, 53, 256, 10)
SEMC mid theta 2 TPSim (53, 45) (53, 45) initial data (4, 53, 256, 10)
SEMC mid theta 3 TPSim (53, 45) (53, 45) initial data (4, 53, 256, 10)
SEMC mid phase_ds 2s
256 426 171
SEMC mid theta 0 TPSim (53, 45) (53, 45) initial data (4, 53, 171, 10)
SEMC mid theta 1 TPSim (53, 45) (53, 45) initial data (4, 53, 171, 10)
SEMC mid theta 2 TPSim (53, 45) (53, 45) initial data (4, 53, 171, 10)
SEMC mid theta 3 TPSim (53, 45) (53, 45) initial data (4, 53, 171, 10)
SEMC mid phase_None 3s
1024 2560 1537
SEMC mid theta 0 TPSim (53, 45) (53,

LEFC high theta 1 TPSim (27, 136) (27, 136) initial data (4, 27, 256, 17)
LEFC high theta 2 TPSim (27, 136) (27, 136) initial data (4, 27, 256, 17)
LEFC high theta 3 TPSim (27, 136) (27, 136) initial data (4, 27, 256, 17)
LEFC high phase_ds 2s
256 426 171
LEFC high theta 0 TPSim (27, 136) (27, 136) initial data (4, 27, 171, 17)
LEFC high theta 1 TPSim (27, 136) (27, 136) initial data (4, 27, 171, 17)
LEFC high theta 2 TPSim (27, 136) (27, 136) initial data (4, 27, 171, 17)
LEFC high theta 3 TPSim (27, 136) (27, 136) initial data (4, 27, 171, 17)
LEFC high phase_None 3s
1024 2560 1537
LEFC high theta 0 TPSim (27, 136) (27, 136) initial data (4, 27, 1537, 17)
LEFC high theta 1 TPSim (27, 136) (27, 136) initial data (4, 27, 1537, 17)
LEFC high theta 2 TPSim (27, 136) (27, 136) initial data (4, 27, 1537, 17)
LEFC high theta 3 TPSim (27, 136) (27, 136) initial data (4, 27, 1537, 17)
LEFC high phase_None 2s
1536 2560 1025
LEFC high theta 0 TPSim (27, 136) (27, 136) initial data (4, 27, 1025,

#### Compute linear regression by elec for all combinations

In [8]:
from utils import rename_elecs
import statsmodels.api as sm
from mne.stats import bonferroni_correction, fdr_correction
import pandas as pd

st = study('Olfacto')
path_tps = join(st.path, 'feature/TPSim_3groups_Enc_phase/')
tps_form = join(path_tps, 'TPS_{}_v=1_elecs=all_ds_sm/'+\
            'TPS_pears_{}_{}_{}_theta{}_data={}_win={}.npz') #su,cond,meth, freq
df_path = join(path_tps, 'LinReg_stats_theta_v=1_elecs=all_ds_sm/')
df_name = join(df_path, 'df_{}_theta{}_data={}_win={}.csv') #meth,f,data_c,win

nfreqs, wins, data_types = 4, ['2s', '3s'], ['phase_None','phase_ds','phase_smooth']
meth, conds, stat = 'btw', ['low','mid','high'], 'LinReg'
subjects = ['CHAF','VACJ','PIRJ','SEMC','FERJ','LEFC']
rois_sel = ['aHC','MFG','IFG','Amg','pPirT','PHG','Ins_olf','OFC_olf']
dict_perf = {'low':1, 'mid':2, 'high':3}

if not exists(df_path):
    makedirs(df_path)

for f,win,data_c in product(range(nfreqs),wins,data_types):
    print('>>> processing theta', f, win, data_c)
    subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
    channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
    tps_scores, T_vals_c, p_vals_c = np.array([]), np.array([]), np.array([])
    p_fdr_c, p_bf_c = np.array([]), np.array([])

    for su in subjects:
        #load all elec info,rename and select electrodes id
        mat = np.load(tps_form.format(meth,su,conds[0],meth,f,data_c,win),allow_pickle=True)
        labels, channels = mat['label'], mat['channel']
        print(su,labels.shape, mat['tps'].shape,np.unique(labels))
        x, y, z = mat['xyz'][:,0], mat['xyz'][:,1], mat['xyz'][:,2]
        labels_new = rename_elecs(labels,x,y,z)
        idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
        labels, channels = labels_new[idx_sel], channels[idx_sel]
        x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]
            
        if stat == 'LinReg':
            tps_su = np.zeros((len(idx_sel),len(conds)))
            all_tps, all_scores = [], []
            for c,cond in enumerate(conds):
                mat = np.load(tps_form.format(meth,su,cond,meth,f,data_c,win))
                tps = mat['tps'][idx_sel]
                nelecs, ntrials = tps.shape
                score_ = np.array([dict_perf[cond]]*ntrials)
                tps_su[:,c] += np.mean(tps,axis=1)
                #fill tps and score vectors
                all_tps.append(tps)
                all_scores.append(score_)

            #compute stats Ttests-unpaired
            all_tps = np.concatenate(all_tps,axis=1)
            all_scores = np.concatenate(all_scores,axis=0)
            T, unc_p = [], []
            for elec in range(nelecs):
                #Tval,pval = stats.kendalltau(all_tps[elec],all_scores)
                Y, X = np.array(all_tps[elec]), sm.add_constant(np.array(all_scores))
                model_ols = sm.OLS(Y,X).fit()
                Tval, pval = np.round(model_ols.tvalues[1],3),model_ols.pvalues[1]
                T.append(Tval), unc_p.append(pval)
        if stat == 'Ttest':
            tps0 = np.load(tps_form.format(su,conds[0],freq,width,step))['tps'][idx_sel]
            nelecs = len(idx_sel)
            tps1 = np.load(tps_form.format(su,conds[1],freq,width,step))['tps'][idx_sel]
            mean0, mean1 = np.mean(tps0,axis=1)[:,np.newaxis], np.mean(tps1,axis=1)[:,np.newaxis]
            tps_su = np.concatenate((mean0,mean1),axis=1)
            #compute stats Ttests-unpaired
            tps0, tps1 = tps0.swapaxes(0,1), tps1.swapaxes(0,1) #ntrials x nelecs
            T, unc_p = ttest_ind(tps0, tps1, equal_var=False)
            print(T.shape, unc_p.shape)
        _, p_fdr = fdr_correction(unc_p)
        _, p_bf = bonferroni_correction(unc_p)

        #fill all df data
        subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
        elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
        labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
        channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
        tps_scores = np.concatenate((tps_scores,tps_su),axis=0) if np.size(tps_scores) else tps_su
        x_c = np.hstack((x_c,x)) if np.size(x_c) else x
        y_c = np.hstack((y_c,y)) if np.size(y_c) else y
        z_c = np.hstack((z_c,z)) if np.size(z_c) else z
        T_vals_c = np.hstack((T_vals_c,T)) if np.size(T_vals_c) else T
        p_vals_c = np.hstack((p_vals_c,unc_p)) if np.size(p_vals_c) else unc_p
        p_fdr_c = np.hstack((p_fdr_c,p_fdr)) if np.size(p_fdr_c) else p_fdr
        p_bf_c = np.hstack((p_bf_c,p_bf)) if np.size(p_bf_c) else p_bf

    data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
                channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
                z_c[:,np.newaxis],elecs_c[:,np.newaxis],tps_scores,T_vals_c[:,np.newaxis],
                p_vals_c[:,np.newaxis],p_fdr_c[:,np.newaxis],p_bf_c[:,np.newaxis]),
                axis=1)
    df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z',
                            'elecs_num', 'tps_'+conds[0], 'tps_'+conds[1],'tps_'+conds[2],
                                    'Tvals', 'unc_p','fdr_p', 'bonf_p'])
    #df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z',
    #                        'elecs_num', 'tps_'+conds[0], 'tps_'+conds[1],
    #                                'Tvals', 'unc_p','fdr_p', 'bonf_p'])
    print(df.shape)
    df.to_csv(df_name.format(meth,f,data_c,win),index=False)   

-> Olfacto loaded
>>> processing theta 0 2s phase_None
CHAF (61,) (61, 10) ['ACC' 'MFG' 'OFC_olf' 'SFG']
VACJ (39,) (39, 36) ['ACC' 'Amg' 'IFG' 'MFG' 'OFC_olf' 'aHC' 'pPirT']
PIRJ (18,) (18, 91) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
SEMC (53,) (53, 210) ['ACC' 'IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC']
FERJ (32,) (32, 120) ['Amg' 'IFG' 'Ins_olf' 'aHC' 'pPirT']
LEFC (27,) (27, 253) ['IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC' 'pPirT']
(164, 14)
>>> processing theta 0 2s phase_ds
CHAF (61,) (61, 10) ['ACC' 'MFG' 'OFC_olf' 'SFG']
VACJ (39,) (39, 36) ['ACC' 'Amg' 'IFG' 'MFG' 'OFC_olf' 'aHC' 'pPirT']
PIRJ (18,) (18, 91) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
SEMC (53,) (53, 210) ['ACC' 'IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC']
FERJ (32,) (32, 120) ['Amg' 'IFG' 'Ins_olf' 'aHC' 'pPirT']
LEFC (27,) (27, 253) ['IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC' 'pPirT']
(164, 14)
>>> processing theta 0 2s phase_smooth
CHAF (61,) (61, 10) ['ACC' 'MFG' 'OFC_olf' 'SFG']
VACJ (39,) (39, 36) ['ACC' 'Amg' 'IFG' 'MFG' 'OFC_olf' 'aHC' 'pPirT

PIRJ (18,) (18, 91) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
SEMC (53,) (53, 210) ['ACC' 'IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC']
FERJ (32,) (32, 120) ['Amg' 'IFG' 'Ins_olf' 'aHC' 'pPirT']
LEFC (27,) (27, 253) ['IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC' 'pPirT']
(164, 14)
>>> processing theta 3 3s phase_None
CHAF (61,) (61, 10) ['ACC' 'MFG' 'OFC_olf' 'SFG']
VACJ (39,) (39, 36) ['ACC' 'Amg' 'IFG' 'MFG' 'OFC_olf' 'aHC' 'pPirT']
PIRJ (18,) (18, 91) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
SEMC (53,) (53, 210) ['ACC' 'IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC']
FERJ (32,) (32, 120) ['Amg' 'IFG' 'Ins_olf' 'aHC' 'pPirT']
LEFC (27,) (27, 253) ['IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC' 'pPirT']
(164, 14)
>>> processing theta 3 3s phase_ds
CHAF (61,) (61, 10) ['ACC' 'MFG' 'OFC_olf' 'SFG']
VACJ (39,) (39, 36) ['ACC' 'Amg' 'IFG' 'MFG' 'OFC_olf' 'aHC' 'pPirT']
PIRJ (18,) (18, 91) ['IFG' 'Ins_olf' 'OFC_olf' 'PHG' 'aHC']
SEMC (53,) (53, 210) ['ACC' 'IFG' 'MFG' 'OFC_olf' 'SFG' 'aHC']
FERJ (32,) (32, 120) ['Amg' 'IFG' 'Ins_olf' 'aHC' '

#### Sum up signif elecs for all combinations of data

In [21]:
from collections import Counter

st = study('Olfacto')
stat = 'LinReg'#'Enc'
meth = 'btw'
olf_regions = ['Amg','pPirT','OFC_olf']
nfreqs, wins, data_types = 4, ['3s', '2s'], ['phase_ds']#,'xpow_ds','xpow_smooth']
thrs = [0.05]
corrections = ['unc_p']#['fdr_p'] unc_p

##################################################################################
path_pow = join(st.path, 'feature/TPSim_3groups_Enc_phase/LinReg_stats_theta_v=1_elecs=all_ds_sm/')
df_name = join(path_pow, 'df_{}_theta{}_data={}_win={}.csv')
df_sig = join(path_pow, 'df_sig_{}_theta{}_roi={}_sign={}_stat={}_data={}_win={}.csv')
##################################################################################

for f,data_c,win in product(range(nfreqs),data_types,wins):
    print('')
    print('processing >>>', 'theta'+str(f),data_c,win)
    df = pd.read_csv(df_name.format(meth,str(f),data_c,win))
    df['labels'] = ['pPirT' if lab == 'Amg' else lab for lab in df['labels']]
    #print(df[df['labels']=='OFC_olf'].mean())

    for th, corr in product(thrs,corrections):
        df_sel = df.loc[df[corr]<th] #
        df_sel['sign'] = ['separation' if t > 0 else 'completion' for t in df_sel['Tvals']]
        #print(df_sel.groupby(['subjects','labels',]).sum()['Tvals'])
        
        rois = np.unique(df_sel['labels'])
        for roi in rois:
            df_roi = df_sel.loc[df_sel['labels']==roi]
            df_inc = df_roi.loc[df_roi['sign']=='completion'].groupby(['subjects']).count()
            df_dec = df_roi.loc[df_roi['sign']=='separation'].groupby(['subjects']).count()
            
            if (df_inc.shape[0] >= 3) or (df_inc.shape[0] >=2 and roi in olf_regions):
                print(roi, 'NB of subjects with completion',df_inc.shape[0],' subjects')
                df_plot = df_roi.loc[df_roi['sign']=='completion']
                print('#electrodes in total >>> ',df_plot.shape[0])
                df_plot.to_csv(df_sig.format(meth,str(f),roi,'compl',corr,data_c,win))

            if (df_dec.shape[0] >= 3) or (df_dec.shape[0] >=2 and roi in olf_regions):
                print(roi, 'NB of subjects with separation',df_dec.shape[0],' subjects')
                df_plot = df_roi.loc[df_roi['sign']=='separation']
                print('#electrodes in total >>> ',df_plot.shape[0])
                df_plot.to_csv(df_sig.format(meth,str(f),roi,'sep',corr,data_c,win))

-> Olfacto loaded

processing >>> theta0 phase_ds 3s
OFC_olf NB of subjects with completion 2  subjects
#electrodes in total >>>  2

processing >>> theta0 phase_ds 2s
OFC_olf NB of subjects with completion 3  subjects
#electrodes in total >>>  3

processing >>> theta1 phase_ds 3s
OFC_olf NB of subjects with completion 2  subjects
#electrodes in total >>>  2

processing >>> theta1 phase_ds 2s
OFC_olf NB of subjects with completion 3  subjects
#electrodes in total >>>  3

processing >>> theta2 phase_ds 3s

processing >>> theta2 phase_ds 2s
OFC_olf NB of subjects with completion 4  subjects
#electrodes in total >>>  4

processing >>> theta3 phase_ds 3s

processing >>> theta3 phase_ds 2s
OFC_olf NB of subjects with completion 3  subjects
#electrodes in total >>>  3


### Mixed models

In [20]:
import numpy as np
from scipy.stats import wilcoxon
from os.path import join
from utils import subjects
import pandas as pd
from brainpipe.system import study

import statsmodels.api as sm
import statsmodels.formula.api as smf

###############################################################################
st = study('Olfacto')
rep_study = join(st.path, 'feature/TPSim_3groups_Enc_phase/')
data_rep = join(rep_study, 'LinReg_stats_theta_v=1_elecs=all_ds_sm/')
csv_file = join(data_rep, 'df_btw_theta{}_data={}_win={}.csv')
###############################################################################
conds = ['low','mid','high']
var_con = {'low':1,'mid':2,'high':3}
rois = ['OFC_olf']#,'aHC','pPirT']
data_types = ['phase_smooth']

for f,data_c,win in product(range(4),data_types,wins):
    df = pd.read_csv(csv_file.format(str(f),data_c,win))
    df['labels'] = ['pPirT' if lab =='Amg' else lab for lab in df['labels']]
    df['labels_RL'] = [x+'_R' if df['x'][i] > 0 else x+'_L' for i,x in enumerate(df['labels'])]

    pvals, rois_sel = [], []
    for roi in rois:
        df_roi = df.loc[df['labels']==roi]
        rois_sel.append(roi)

        df_take = df_roi
        data = np.concatenate([df_take['tps_'+cond] for cond in conds])
        richness = np.concatenate([[var_con[cond]]*len(df_take['tps_'+cond]) for cond in conds])
        subjs = np.concatenate([df_take['subjects']]*len(conds))
        all_df = np.concatenate((subjs[:,np.newaxis],richness[:,np.newaxis],
                                 data[:,np.newaxis]),axis=-1)
        df_stats = pd.DataFrame(all_df, columns=['subjs','richness','tps'])
        df_stats['richness'] = df_stats['richness'].astype('float')
        df_stats['tps'] = df_stats['tps'].astype('float')

        print(">>> processing ", f, data_c, win, roi)
        print(df_take[['tps_low','tps_mid','tps_high']].mean().values)
        md = smf.mixedlm("tps ~ richness", df_stats, groups = df_stats['subjs'])
        mdf = md.fit()
        pvals.append(mdf.pvalues[1])
        print(mdf.pvalues[1])

-> Olfacto loaded
>>> processing  0 phase_smooth 3s OFC_olf
[0.98454482 0.99489273 0.98710432]
0.8464053357126909
>>> processing  0 phase_smooth 2s OFC_olf
[0.98529369 0.99823642 0.98465524]
0.9652009616858914
>>> processing  1 phase_smooth 3s OFC_olf
[0.98378569 1.01649967 0.98354656]


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.p

0.9866787482861265
>>> processing  1 phase_smooth 2s OFC_olf
[0.97578241 1.01549841 0.96898603]
0.7189991942137233
>>> processing  2 phase_smooth 3s OFC_olf
[0.98701009 0.99748889 0.99196762]


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


0.6739711890869728
>>> processing  2 phase_smooth 2s OFC_olf
[0.98194839 1.00948333 0.98376862]
0.8944067406064321
>>> processing  3 phase_smooth 3s OFC_olf
[0.98266646 1.01301057 0.98604929]


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2063: ConvergenceWarning: MixedLM opt

0.7993688049391738
>>> processing  3 phase_smooth 2s OFC_olf
[0.97557358 1.01715192 0.98086626]
0.7614444844370851


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
